In [93]:
import requests
import pandas
import numpy
from arcgis.features import GeoAccessor
from arcpy.sa import *
import arcpy
import random

In [74]:
hour = 12 #Hour to retrieve weather data for, in 24 hour time. Midnight uses a value of 24 for the previous day.

#Date
year = 2024
month = 3
day = 25

In [85]:
#File paths
stem_path = r'C:/Users/tjjoh/Documents/GIS5571'

gdb_path = stem_path + r'/FinalProject_GDB.gdb'
temp_fc_path = gdb_path + r'/temp'
humid_fc_path = gdb_path + r'/humid'
pres_fc_path = gdb_path + r'/pres'
wind_fc_path = gdb_path + r'/wind'

In [75]:
utc = str((hour + 5) % 24).zfill(2) #Convert time to UTC. This is valid only during daylight savings time

#Convert date and time to strings
yr = str(year)
mn = str(month).zfill(2)
dy_ndawn = str(day).zfill(2)
if hour < 19:
    dy_noaa = str(day).zfill(2)
else:
    dy_noaa = str(day + 1).zfill(2)
hr = str(hour).zfill(2)

#URLs for API calls
noaa_url = r'https://api.synopticlabs.org/v2/stations/nearesttime?token=d8c6aee36a994f90857925cea26934be&bbox=-105%2C42%2C-86.5%2C49.5&minmax=1&minmaxtype=local&units=temp%7Cf%2Cspeed%7Cmph&attime=' + yr + mn + dy_noaa + utc + '00&within=90&status=active&qc_checks=all'
ndawn_url = 'https://ndawn.ndsu.nodak.edu/table.csv?station>0&variable=hdt&variable=hdbp&variable=hdrh&variable=hdws&variable=hdwd&ttype=hourly&quick_pick=&begin_date=' + yr + '-' + mn + '-' + dy_ndawn + '&end_date=' + yr + '-' + mn + '-' + dy_ndawn

**Prepare NOAA Data**

In [76]:
#Make API request
noaa_response = requests.get(noaa_url)

In [77]:
#Convert NOAA data to dataframe
noaa_df = pandas.json_normalize(noaa_response.json()['STATION'])

In [78]:
#Rename columns
noaa_df = noaa_df.rename(columns = {'OBSERVATIONS.air_temp_value_1.value' : 'Temperature',
                                    'OBSERVATIONS.relative_humidity_value_1.value' : 'Humidity',
                                    'OBSERVATIONS.wind_speed_value_1.value' : 'WindSpeed',
                                    'OBSERVATIONS.wind_direction_value_1.value' : 'WindDirection',
                                    'OBSERVATIONS.altimeter_value_1.value' : 'Pressure',
                                    'NAME' : 'Name',
                                    'ELEVATION' : 'Elevation',
                                    'LATITUDE' : 'Latitude',
                                    'LONGITUDE' : 'Longitude',
                                    'STATE' : 'State',
                                    'TIMEZONE' : 'Timezone'
                                   })

In [79]:
#Keep only desired columns
noaa_df = noaa_df[['ID','Name','Elevation','Latitude','Longitude','State','Timezone',
                   'Temperature','Humidity','WindSpeed','WindDirection','Pressure'
                  ]]

In [80]:
#Convert pressure units to mbar, the same units that NDAWN uses
noaa_df['Pressure'] = noaa_df['Pressure'] / 100

**Prepare NDAWN Data**

In [81]:
#Read data into a pandas dataframe
ndawn_df = pandas.read_csv(ndawn_url, skiprows = [0,1,2,4])

In [82]:
#Choose only the specified hour
ndawn_df = ndawn_df[ndawn_df['Hour'] == hour * 100]

In [83]:
#Rename columns
ndawn_df = ndawn_df.rename(columns = {'Station Name' : 'Name',
                                      'Avg Air Temp' : 'Temperature',
                                      'Avg Baro Press' : 'Pressure',
                                      'Avg Rel Hum' : 'Humidity',
                                      'Avg Wind Speed' : 'WindSpeed',
                                      'Avg Wind Dir' : 'WindDirection'
                                     })

In [84]:
#Keep only desired columns
ndawn_df = ndawn_df[['Name','Elevation','Latitude','Longitude',
                     'Temperature','Humidity','WindSpeed','WindDirection','Pressure'
                    ]]

**Combine Data Frames**

In [86]:
#Concatenate data frames
weather_df = pandas.concat([noaa_df, ndawn_df], ignore_index = True)

In [87]:
#Convert to a spatially-enabled dataframe
weather_sedf = GeoAccessor.from_xy(weather_df, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

In [88]:
#Calculate z-scores for weather quantities
weather_sedf['Temp_z'] = abs((weather_sedf['Temperature'] - weather_sedf['Temperature'].mean()) / weather_sedf['Temperature'].std())
weather_sedf['Humid_z'] = abs((weather_sedf['Humidity'] - weather_sedf['Humidity'].mean()) / weather_sedf['Humidity'].std())
weather_sedf['Pres_z'] = abs((weather_sedf['Pressure'] - weather_sedf['Pressure'].mean()) / weather_sedf['Pressure'].std())
weather_sedf['Wind_z'] = abs((weather_sedf['WindSpeed'] - weather_sedf['WindSpeed'].mean()) / weather_sedf['WindSpeed'].std())

In [89]:
#Remove instances with high z-score because they are likely inaccurate
temp_sedf = weather_sedf[weather_sedf['Temp_z'] < 3]
humid_sedf = weather_sedf[weather_sedf['Humid_z'] < 3]
pres_sedf = weather_sedf[weather_sedf['Pres_z'] < 3]
wind_sedf = weather_sedf[weather_sedf['Wind_z'] < 3]

In [90]:
#Deconstruct wind into North and East components
wind_sedf = wind_sedf[wind_sedf['WindDirection'].notna()]

wind_sedf['Wind_N'] = wind_sedf['WindSpeed'] * numpy.cos(numpy.radians(wind_sedf['WindDirection']))
wind_sedf['Wind_E'] = wind_sedf['WindSpeed'] * numpy.sin(numpy.radians(wind_sedf['WindDirection']))

In [91]:
#Convert spatially enabled dataframes to feature classes
temp_fc = temp_sedf.spatial.to_featureclass(location = temp_fc_path)
humid_fc = humid_sedf.spatial.to_featureclass(location = humid_fc_path)
pres_fc = pres_sedf.spatial.to_featureclass(location = pres_fc_path)
wind_fc = wind_sedf.spatial.to_featureclass(location = wind_fc_path)

**Leave One Out Cross Validation**

*Temperature*

In [94]:
input_fc = temp_fc_path
errors_idw1 = []
errors_idw2 = []
errors_nn = []
errors_spline = []
errors_kriging = []

# Get all points
points = []
with arcpy.da.SearchCursor(input_fc, ["SHAPE@XY", "temperature"]) as cursor:
    for row in cursor:
        points.append((row[0], row[1]))

# Randomly sample 50 points
random.seed(42)  # Set seed for reproducibility
sampled_points = random.sample(points, 50)

In [95]:
# Loop over each sampled point, leaving one out each time
for i, (point, actual_value) in enumerate(sampled_points):
    # Create a temporary feature class excluding the current point
    temp_fc = "memory\\temp_points"
    arcpy.management.CreateFeatureclass("memory", "temp_points", "POINT", spatial_reference=input_fc)
    
    # Add the "temperature" field to the temporary feature class
    arcpy.management.AddField(temp_fc, "temperature", "DOUBLE")
    
    # Insert all sampled points except the current one
    with arcpy.da.InsertCursor(temp_fc, ["SHAPE@XY", "temperature"]) as insert_cursor:
        for j, (other_point, other_value) in enumerate(sampled_points):
            if i != j:
                insert_cursor.insertRow([other_point, other_value])
    
    # Perform IDW interpolation with power set to 1
    output_raster_idw1 = arcpy.sa.Idw(temp_fc, "temperature", power=1.0)
    
    # Perform IDW interpolation with power set to 2
    output_raster_idw2 = arcpy.sa.Idw(temp_fc, "temperature", power=2.0)
    
    # Perform Natural Neighbor interpolation
    output_raster_nn = arcpy.sa.NaturalNeighbor(temp_fc, "temperature")
    
    # Perform Spline interpolation
    output_raster_spline = arcpy.sa.Spline(temp_fc, "temperature")
    
    # Perform Kriging interpolation with spherical model
    output_raster_kriging = arcpy.sa.Kriging(temp_fc, "temperature", kriging_model="SPHERICAL")
    
    # Extract the raster value at the current point's location
    point_fc = "memory\\temp_point"
    arcpy.management.CreateFeatureclass("memory", "temp_point", "POINT", spatial_reference=input_fc)
    
    with arcpy.da.InsertCursor(point_fc, ["SHAPE@XY"]) as point_cursor:
        point_cursor.insertRow([point])
    
    extracted_values_idw1 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw1, "memory\\extracted_values_idw1")
    extracted_values_idw2 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw2, "memory\\extracted_values_idw2")
    extracted_values_nn = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_nn, "memory\\extracted_values_nn")
    extracted_values_spline = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_spline, "memory\\extracted_values_spline")
    extracted_values_kriging = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_kriging, "memory\\extracted_values_kriging")
    
    with arcpy.da.SearchCursor(extracted_values_idw1, ["RASTERVALU"]) as cursor_idw1, \
         arcpy.da.SearchCursor(extracted_values_idw2, ["RASTERVALU"]) as cursor_idw2, \
         arcpy.da.SearchCursor(extracted_values_nn, ["RASTERVALU"]) as cursor_nn, \
         arcpy.da.SearchCursor(extracted_values_spline, ["RASTERVALU"]) as cursor_spline, \
         arcpy.da.SearchCursor(extracted_values_kriging, ["RASTERVALU"]) as cursor_kriging:

        for raster_row_idw1, raster_row_idw2, raster_row_nn, raster_row_spline, raster_row_kriging in zip(cursor_idw1, cursor_idw2, cursor_nn, cursor_spline, cursor_kriging):
            interpolated_value_idw1 = raster_row_idw1[0]
            interpolated_value_idw2 = raster_row_idw2[0]
            interpolated_value_nn = raster_row_nn[0]
            interpolated_value_spline = raster_row_spline[0]
            interpolated_value_kriging = raster_row_kriging[0]
            if None not in (interpolated_value_idw1, interpolated_value_idw2, interpolated_value_nn, interpolated_value_spline, interpolated_value_kriging):
                # Calculate the error for each method
                error_idw1 = interpolated_value_idw1 - actual_value
                error_idw2 = interpolated_value_idw2 - actual_value
                error_nn = interpolated_value_nn - actual_value
                error_spline = interpolated_value_spline - actual_value
                error_kriging = interpolated_value_kriging - actual_value
                errors_idw1.append(error_idw1)
                errors_idw2.append(error_idw2)
                errors_nn.append(error_nn)
                errors_spline.append(error_spline)
                errors_kriging.append(error_kriging)
                # Print progress
                print(f"Iteration {i+1}/{len(sampled_points)} complete. IDW1 Error: {error_idw1}, IDW2 Error: {error_idw2}, NN Error: {error_nn}, Spline Error: {error_spline}, Kriging Error: {error_kriging}")
            else:
                print(f"Iteration {i+1}/{len(sampled_points)} complete. Interpolated value is None for one of the methods, skipping this point.")

# Calculate RMSE for all methods
if errors_idw1:
    rmse_idw1 = numpy.sqrt(numpy.mean(numpy.square(errors_idw1)))
    print(f"IDW1 RMSE: {rmse_idw1}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW1.")

if errors_idw2:
    rmse_idw2 = numpy.sqrt(numpy.mean(numpy.square(errors_idw2)))
    print(f"IDW2 RMSE: {rmse_idw2}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW2.")

if errors_nn:
    rmse_nn = numpy.sqrt(numpy.mean(numpy.square(errors_nn)))
    print(f"Natural Neighbor RMSE: {rmse_nn}")
else:
    print("No valid interpolated values were found to calculate RMSE for Natural Neighbor.")

if errors_spline:
    rmse_spline = numpy.sqrt(numpy.mean(numpy.square(errors_spline)))
    print(f"Spline RMSE: {rmse_spline}")
else:
    print("No valid interpolated values were found to calculate RMSE for Spline.")

if errors_kriging:
    rmse_kriging = numpy.sqrt(numpy.mean(numpy.square(errors_kriging)))
    print(f"Kriging RMSE: {rmse_kriging}")
else:
    print("No valid interpolated values were found to calculate RMSE for Kriging.")

Iteration 1/50 complete. IDW1 Error: -0.41579360961914347, IDW2 Error: -0.7652236938476591, NN Error: 5.814896392822263, Spline Error: -0.1422836303710966, Kriging Error: 3.309948730468747
Iteration 2/50 complete. IDW1 Error: 4.0467918395996065, IDW2 Error: 2.7737930297851534, NN Error: 5.114716339111325, Spline Error: 4.149411010742185, Kriging Error: 4.431481170654294
Iteration 3/50 complete. Interpolated value is None for one of the methods, skipping this point.
Iteration 4/50 complete. Interpolated value is None for one of the methods, skipping this point.
Iteration 5/50 complete. IDW1 Error: 4.354824523925778, IDW2 Error: 1.2026829528808562, NN Error: -0.5644869995117219, Spline Error: -0.6868739318847688, Kriging Error: -0.9469599914550813
Iteration 6/50 complete. IDW1 Error: 7.297493896484376, IDW2 Error: 4.769505462646485, NN Error: 3.9460572814941415, Spline Error: 2.4381951904296884, Kriging Error: 3.5976457214355477
Iteration 7/50 complete. IDW1 Error: -7.348962402343751, ID

Iteration 49/50 complete. IDW1 Error: -1.7255065917968722, IDW2 Error: -0.8102233886718722, NN Error: -0.9797447204589815, Spline Error: -0.9287574768066378, Kriging Error: -0.9642684936523409
Iteration 50/50 complete. IDW1 Error: 4.946464538574219, IDW2 Error: 1.4924125671386719, NN Error: 0.9491920471191406, Spline Error: -1.4967689514160156, Kriging Error: 0.5200996398925781
IDW1 RMSE: 3.54600811013907
IDW2 RMSE: 2.654941521418057
Natural Neighbor RMSE: 2.7438210674159254
Spline RMSE: 4.704541217729637
Kriging RMSE: 2.5040613857555973


*Humidity*

In [96]:
input_fc = humid_fc_path
errors_idw1 = []
errors_idw2 = []
errors_nn = []
errors_spline = []
errors_kriging = []

# Get all points
points = []
with arcpy.da.SearchCursor(input_fc, ["SHAPE@XY", "humidity"]) as cursor:
    for row in cursor:
        points.append((row[0], row[1]))

# Randomly sample 50 points
random.seed(42)  # Set seed for reproducibility
sampled_points = random.sample(points, 50)

In [97]:
# Loop over each sampled point, leaving one out each time
for i, (point, actual_value) in enumerate(sampled_points):
    # Create a temporary feature class excluding the current point
    temp_fc = "memory\\temp_points"
    arcpy.management.CreateFeatureclass("memory", "temp_points", "POINT", spatial_reference=input_fc)
    
    # Add the "humidity" field to the temporary feature class
    arcpy.management.AddField(temp_fc, "humidity", "DOUBLE")
    
    # Insert all sampled points except the current one
    with arcpy.da.InsertCursor(temp_fc, ["SHAPE@XY", "humidity"]) as insert_cursor:
        for j, (other_point, other_value) in enumerate(sampled_points):
            if i != j:
                insert_cursor.insertRow([other_point, other_value])
    
    # Perform IDW interpolation with power set to 1
    output_raster_idw1 = arcpy.sa.Idw(temp_fc, "humidity", power=1.0)
    
    # Perform IDW interpolation with power set to 2
    output_raster_idw2 = arcpy.sa.Idw(temp_fc, "humidity", power=2.0)
    
    # Perform Natural Neighbor interpolation
    output_raster_nn = arcpy.sa.NaturalNeighbor(temp_fc, "humidity")
    
    # Perform Spline interpolation
    output_raster_spline = arcpy.sa.Spline(temp_fc, "humidity")
    
    # Perform Kriging interpolation with spherical model
    output_raster_kriging = arcpy.sa.Kriging(temp_fc, "humidity", kriging_model="SPHERICAL")
    
    # Extract the raster value at the current point's location
    point_fc = "memory\\temp_point"
    arcpy.management.CreateFeatureclass("memory", "temp_point", "POINT", spatial_reference=input_fc)
    
    with arcpy.da.InsertCursor(point_fc, ["SHAPE@XY"]) as point_cursor:
        point_cursor.insertRow([point])
    
    extracted_values_idw1 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw1, "memory\\extracted_values_idw1")
    extracted_values_idw2 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw2, "memory\\extracted_values_idw2")
    extracted_values_nn = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_nn, "memory\\extracted_values_nn")
    extracted_values_spline = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_spline, "memory\\extracted_values_spline")
    extracted_values_kriging = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_kriging, "memory\\extracted_values_kriging")
    
    with arcpy.da.SearchCursor(extracted_values_idw1, ["RASTERVALU"]) as cursor_idw1, \
         arcpy.da.SearchCursor(extracted_values_idw2, ["RASTERVALU"]) as cursor_idw2, \
         arcpy.da.SearchCursor(extracted_values_nn, ["RASTERVALU"]) as cursor_nn, \
         arcpy.da.SearchCursor(extracted_values_spline, ["RASTERVALU"]) as cursor_spline, \
         arcpy.da.SearchCursor(extracted_values_kriging, ["RASTERVALU"]) as cursor_kriging:

        for raster_row_idw1, raster_row_idw2, raster_row_nn, raster_row_spline, raster_row_kriging in zip(cursor_idw1, cursor_idw2, cursor_nn, cursor_spline, cursor_kriging):
            interpolated_value_idw1 = raster_row_idw1[0]
            interpolated_value_idw2 = raster_row_idw2[0]
            interpolated_value_nn = raster_row_nn[0]
            interpolated_value_spline = raster_row_spline[0]
            interpolated_value_kriging = raster_row_kriging[0]
            if None not in (interpolated_value_idw1, interpolated_value_idw2, interpolated_value_nn, interpolated_value_spline, interpolated_value_kriging):
                # Calculate the error for each method
                error_idw1 = interpolated_value_idw1 - actual_value
                error_idw2 = interpolated_value_idw2 - actual_value
                error_nn = interpolated_value_nn - actual_value
                error_spline = interpolated_value_spline - actual_value
                error_kriging = interpolated_value_kriging - actual_value
                errors_idw1.append(error_idw1)
                errors_idw2.append(error_idw2)
                errors_nn.append(error_nn)
                errors_spline.append(error_spline)
                errors_kriging.append(error_kriging)
                # Print progress
                print(f"Iteration {i+1}/{len(sampled_points)} complete. IDW1 Error: {error_idw1}, IDW2 Error: {error_idw2}, NN Error: {error_nn}, Spline Error: {error_spline}, Kriging Error: {error_kriging}")
            else:
                print(f"Iteration {i+1}/{len(sampled_points)} complete. Interpolated value is None for one of the methods, skipping this point.")

# Calculate RMSE for all methods
if errors_idw1:
    rmse_idw1 = numpy.sqrt(numpy.mean(numpy.square(errors_idw1)))
    print(f"IDW1 RMSE: {rmse_idw1}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW1.")

if errors_idw2:
    rmse_idw2 = numpy.sqrt(numpy.mean(numpy.square(errors_idw2)))
    print(f"IDW2 RMSE: {rmse_idw2}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW2.")

if errors_nn:
    rmse_nn = numpy.sqrt(numpy.mean(numpy.square(errors_nn)))
    print(f"Natural Neighbor RMSE: {rmse_nn}")
else:
    print("No valid interpolated values were found to calculate RMSE for Natural Neighbor.")

if errors_spline:
    rmse_spline = numpy.sqrt(numpy.mean(numpy.square(errors_spline)))
    print(f"Spline RMSE: {rmse_spline}")
else:
    print("No valid interpolated values were found to calculate RMSE for Spline.")

if errors_kriging:
    rmse_kriging = numpy.sqrt(numpy.mean(numpy.square(errors_kriging)))
    print(f"Kriging RMSE: {rmse_kriging}")
else:
    print("No valid interpolated values were found to calculate RMSE for Kriging.")

Iteration 1/50 complete. IDW1 Error: -2.7593154907226562, IDW2 Error: -0.576934814453125, NN Error: -3.5897979736328125, Spline Error: 19.4034423828125, Kriging Error: -1.6883392333984375
Iteration 2/50 complete. IDW1 Error: 1.3394882202148466, IDW2 Error: -0.7408645629882784, NN Error: -2.1203887939453097, Spline Error: 25.884692382812503, Kriging Error: -2.352116394042966
Iteration 3/50 complete. IDW1 Error: 30.307564697265626, IDW2 Error: 22.32192321777344, NN Error: 3.472836456298829, Spline Error: 1.667588195800782, Kriging Error: 5.35391326904297
Iteration 4/50 complete. IDW1 Error: 6.260894775390625, IDW2 Error: 8.172286987304688, NN Error: 9.333091735839844, Spline Error: 17.424545288085938, Kriging Error: 10.869491577148438
Iteration 5/50 complete. IDW1 Error: -5.4261932373046875, IDW2 Error: -5.06121826171875, NN Error: -3.009063720703125, Spline Error: 0.36712646484375, Kriging Error: -2.6451873779296875
Iteration 6/50 complete. IDW1 Error: -3.0183334350585938, IDW2 Error: -

Iteration 48/50 complete. IDW1 Error: -6.315040588378906, IDW2 Error: -5.355072021484375, NN Error: -5.981742858886719, Spline Error: 9.043685913085938, Kriging Error: -4.611045837402344
Iteration 49/50 complete. IDW1 Error: 9.395451049804691, IDW2 Error: 9.089428405761723, NN Error: 7.557507019042973, Spline Error: 6.4795574951171915, Kriging Error: 8.052586059570316
Iteration 50/50 complete. Interpolated value is None for one of the methods, skipping this point.
IDW1 RMSE: 9.218767267514734
IDW2 RMSE: 7.925466381185673
Natural Neighbor RMSE: 6.899914640406023
Spline RMSE: 23.38557675294933
Kriging RMSE: 7.347880217284561


*Pressure*

In [98]:
input_fc = pres_fc_path
errors_idw1 = []
errors_idw2 = []
errors_nn = []
errors_spline = []
errors_kriging = []

# Get all points
points = []
with arcpy.da.SearchCursor(input_fc, ["SHAPE@XY", "pressure"]) as cursor:
    for row in cursor:
        points.append((row[0], row[1]))

# Randomly sample 50 points
random.seed(42)  # Set seed for reproducibility
sampled_points = random.sample(points, 50)

In [99]:
# Loop over each sampled point, leaving one out each time
for i, (point, actual_value) in enumerate(sampled_points):
    # Create a temporary feature class excluding the current point
    temp_fc = "memory\\temp_points"
    arcpy.management.CreateFeatureclass("memory", "temp_points", "POINT", spatial_reference=input_fc)
    
    # Add the "pressure" field to the temporary feature class
    arcpy.management.AddField(temp_fc, "pressure", "DOUBLE")
    
    # Insert all sampled points except the current one
    with arcpy.da.InsertCursor(temp_fc, ["SHAPE@XY", "pressure"]) as insert_cursor:
        for j, (other_point, other_value) in enumerate(sampled_points):
            if i != j:
                insert_cursor.insertRow([other_point, other_value])
    
    # Perform IDW interpolation with power set to 1
    output_raster_idw1 = arcpy.sa.Idw(temp_fc, "pressure", power=1.0)
    
    # Perform IDW interpolation with power set to 2
    output_raster_idw2 = arcpy.sa.Idw(temp_fc, "pressure", power=2.0)
    
    # Perform Natural Neighbor interpolation
    output_raster_nn = arcpy.sa.NaturalNeighbor(temp_fc, "pressure")
    
    # Perform Spline interpolation
    output_raster_spline = arcpy.sa.Spline(temp_fc, "pressure")
    
    # Perform Kriging interpolation with spherical model
    output_raster_kriging = arcpy.sa.Kriging(temp_fc, "pressure", kriging_model="SPHERICAL")
    
    # Extract the raster value at the current point's location
    point_fc = "memory\\temp_point"
    arcpy.management.CreateFeatureclass("memory", "temp_point", "POINT", spatial_reference=input_fc)
    
    with arcpy.da.InsertCursor(point_fc, ["SHAPE@XY"]) as point_cursor:
        point_cursor.insertRow([point])
    
    extracted_values_idw1 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw1, "memory\\extracted_values_idw1")
    extracted_values_idw2 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw2, "memory\\extracted_values_idw2")
    extracted_values_nn = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_nn, "memory\\extracted_values_nn")
    extracted_values_spline = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_spline, "memory\\extracted_values_spline")
    extracted_values_kriging = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_kriging, "memory\\extracted_values_kriging")
    
    with arcpy.da.SearchCursor(extracted_values_idw1, ["RASTERVALU"]) as cursor_idw1, \
         arcpy.da.SearchCursor(extracted_values_idw2, ["RASTERVALU"]) as cursor_idw2, \
         arcpy.da.SearchCursor(extracted_values_nn, ["RASTERVALU"]) as cursor_nn, \
         arcpy.da.SearchCursor(extracted_values_spline, ["RASTERVALU"]) as cursor_spline, \
         arcpy.da.SearchCursor(extracted_values_kriging, ["RASTERVALU"]) as cursor_kriging:

        for raster_row_idw1, raster_row_idw2, raster_row_nn, raster_row_spline, raster_row_kriging in zip(cursor_idw1, cursor_idw2, cursor_nn, cursor_spline, cursor_kriging):
            interpolated_value_idw1 = raster_row_idw1[0]
            interpolated_value_idw2 = raster_row_idw2[0]
            interpolated_value_nn = raster_row_nn[0]
            interpolated_value_spline = raster_row_spline[0]
            interpolated_value_kriging = raster_row_kriging[0]
            if None not in (interpolated_value_idw1, interpolated_value_idw2, interpolated_value_nn, interpolated_value_spline, interpolated_value_kriging):
                # Calculate the error for each method
                error_idw1 = interpolated_value_idw1 - actual_value
                error_idw2 = interpolated_value_idw2 - actual_value
                error_nn = interpolated_value_nn - actual_value
                error_spline = interpolated_value_spline - actual_value
                error_kriging = interpolated_value_kriging - actual_value
                errors_idw1.append(error_idw1)
                errors_idw2.append(error_idw2)
                errors_nn.append(error_nn)
                errors_spline.append(error_spline)
                errors_kriging.append(error_kriging)
                # Print progress
                print(f"Iteration {i+1}/{len(sampled_points)} complete. IDW1 Error: {error_idw1}, IDW2 Error: {error_idw2}, NN Error: {error_nn}, Spline Error: {error_spline}, Kriging Error: {error_kriging}")
            else:
                print(f"Iteration {i+1}/{len(sampled_points)} complete. Interpolated value is None for one of the methods, skipping this point.")

# Calculate RMSE for all methods
if errors_idw1:
    rmse_idw1 = numpy.sqrt(numpy.mean(numpy.square(errors_idw1)))
    print(f"IDW1 RMSE: {rmse_idw1}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW1.")

if errors_idw2:
    rmse_idw2 = numpy.sqrt(numpy.mean(numpy.square(errors_idw2)))
    print(f"IDW2 RMSE: {rmse_idw2}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW2.")

if errors_nn:
    rmse_nn = numpy.sqrt(numpy.mean(numpy.square(errors_nn)))
    print(f"Natural Neighbor RMSE: {rmse_nn}")
else:
    print("No valid interpolated values were found to calculate RMSE for Natural Neighbor.")

if errors_spline:
    rmse_spline = numpy.sqrt(numpy.mean(numpy.square(errors_spline)))
    print(f"Spline RMSE: {rmse_spline}")
else:
    print("No valid interpolated values were found to calculate RMSE for Spline.")

if errors_kriging:
    rmse_kriging = numpy.sqrt(numpy.mean(numpy.square(errors_kriging)))
    print(f"Kriging RMSE: {rmse_kriging}")
else:
    print("No valid interpolated values were found to calculate RMSE for Kriging.")

Iteration 1/50 complete. IDW1 Error: -0.3961578125000642, IDW2 Error: -0.8219390625000642, NN Error: 0.4076141601561858, Spline Error: 9.244833398437436, Kriging Error: 0.2397064453124358
Iteration 2/50 complete. IDW1 Error: -0.3516323242188264, IDW2 Error: 0.0661533203124236, NN Error: -1.1373989257813264, Spline Error: -3.5160000000000764, Kriging Error: -0.9610683593750764
Iteration 3/50 complete. IDW1 Error: -11.01100898437494, IDW2 Error: -12.54665351562494, NN Error: 0.18216728515631075, Spline Error: 13.23349785156256, Kriging Error: -8.87807441406244
Iteration 4/50 complete. IDW1 Error: -0.290651074218772, IDW2 Error: -0.12219404296877201, NN Error: 0.283018359374978, Spline Error: 0.295652636718728, Kriging Error: 0.08032060546872799
Iteration 5/50 complete. IDW1 Error: 2.9807824218750056, IDW2 Error: 2.3416833007812556, NN Error: 2.0197838867187556, Spline Error: 1.1419152343750056, Kriging Error: 2.2206505859375056
Iteration 6/50 complete. IDW1 Error: 1.7686198242187174, IDW

Iteration 49/50 complete. IDW1 Error: -9.272741992187548, IDW2 Error: -11.685888964843798, NN Error: -10.443701464843798, Spline Error: -18.559973437500048, Kriging Error: -12.313696582031298
Iteration 50/50 complete. IDW1 Error: 0.2487023437500966, IDW2 Error: 0.09507685546884659, NN Error: -0.5532385742186534, Spline Error: 0.4598839843750966, Kriging Error: -0.3329626953124034
IDW1 RMSE: 7.985796297209024
IDW2 RMSE: 8.477178267966798
Natural Neighbor RMSE: 8.812098609267393
Spline RMSE: 15.556991291998326
Kriging RMSE: 8.35874542076633


*Wind (N-S)*

In [100]:
input_fc = wind_fc_path
errors_idw1 = []
errors_idw2 = []
errors_nn = []
errors_spline = []
errors_kriging = []

# Get all points
points = []
with arcpy.da.SearchCursor(input_fc, ["SHAPE@XY", "wind_n"]) as cursor:
    for row in cursor:
        points.append((row[0], row[1]))

# Randomly sample 50 points
random.seed(42)  # Set seed for reproducibility
sampled_points = random.sample(points, 50)

In [101]:
# Loop over each sampled point, leaving one out each time
for i, (point, actual_value) in enumerate(sampled_points):
    # Create a temporary feature class excluding the current point
    temp_fc = "memory\\temp_points"
    arcpy.management.CreateFeatureclass("memory", "temp_points", "POINT", spatial_reference=input_fc)
    
    # Add the "wind_n" field to the temporary feature class
    arcpy.management.AddField(temp_fc, "wind_n", "DOUBLE")
    
    # Insert all sampled points except the current one
    with arcpy.da.InsertCursor(temp_fc, ["SHAPE@XY", "wind_n"]) as insert_cursor:
        for j, (other_point, other_value) in enumerate(sampled_points):
            if i != j:
                insert_cursor.insertRow([other_point, other_value])
    
    # Perform IDW interpolation with power set to 1
    output_raster_idw1 = arcpy.sa.Idw(temp_fc, "wind_n", power=1.0)
    
    # Perform IDW interpolation with power set to 2
    output_raster_idw2 = arcpy.sa.Idw(temp_fc, "wind_n", power=2.0)
    
    # Perform Natural Neighbor interpolation
    output_raster_nn = arcpy.sa.NaturalNeighbor(temp_fc, "wind_n")
    
    # Perform Spline interpolation
    output_raster_spline = arcpy.sa.Spline(temp_fc, "wind_n")
    
    # Perform Kriging interpolation with spherical model
    output_raster_kriging = arcpy.sa.Kriging(temp_fc, "wind_n", kriging_model="SPHERICAL")
    
    # Extract the raster value at the current point's location
    point_fc = "memory\\temp_point"
    arcpy.management.CreateFeatureclass("memory", "temp_point", "POINT", spatial_reference=input_fc)
    
    with arcpy.da.InsertCursor(point_fc, ["SHAPE@XY"]) as point_cursor:
        point_cursor.insertRow([point])
    
    extracted_values_idw1 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw1, "memory\\extracted_values_idw1")
    extracted_values_idw2 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw2, "memory\\extracted_values_idw2")
    extracted_values_nn = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_nn, "memory\\extracted_values_nn")
    extracted_values_spline = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_spline, "memory\\extracted_values_spline")
    extracted_values_kriging = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_kriging, "memory\\extracted_values_kriging")
    
    with arcpy.da.SearchCursor(extracted_values_idw1, ["RASTERVALU"]) as cursor_idw1, \
         arcpy.da.SearchCursor(extracted_values_idw2, ["RASTERVALU"]) as cursor_idw2, \
         arcpy.da.SearchCursor(extracted_values_nn, ["RASTERVALU"]) as cursor_nn, \
         arcpy.da.SearchCursor(extracted_values_spline, ["RASTERVALU"]) as cursor_spline, \
         arcpy.da.SearchCursor(extracted_values_kriging, ["RASTERVALU"]) as cursor_kriging:

        for raster_row_idw1, raster_row_idw2, raster_row_nn, raster_row_spline, raster_row_kriging in zip(cursor_idw1, cursor_idw2, cursor_nn, cursor_spline, cursor_kriging):
            interpolated_value_idw1 = raster_row_idw1[0]
            interpolated_value_idw2 = raster_row_idw2[0]
            interpolated_value_nn = raster_row_nn[0]
            interpolated_value_spline = raster_row_spline[0]
            interpolated_value_kriging = raster_row_kriging[0]
            if None not in (interpolated_value_idw1, interpolated_value_idw2, interpolated_value_nn, interpolated_value_spline, interpolated_value_kriging):
                # Calculate the error for each method
                error_idw1 = interpolated_value_idw1 - actual_value
                error_idw2 = interpolated_value_idw2 - actual_value
                error_nn = interpolated_value_nn - actual_value
                error_spline = interpolated_value_spline - actual_value
                error_kriging = interpolated_value_kriging - actual_value
                errors_idw1.append(error_idw1)
                errors_idw2.append(error_idw2)
                errors_nn.append(error_nn)
                errors_spline.append(error_spline)
                errors_kriging.append(error_kriging)
                # Print progress
                print(f"Iteration {i+1}/{len(sampled_points)} complete. IDW1 Error: {error_idw1}, IDW2 Error: {error_idw2}, NN Error: {error_nn}, Spline Error: {error_spline}, Kriging Error: {error_kriging}")
            else:
                print(f"Iteration {i+1}/{len(sampled_points)} complete. Interpolated value is None for one of the methods, skipping this point.")

# Calculate RMSE for all methods
if errors_idw1:
    rmse_idw1 = numpy.sqrt(numpy.mean(numpy.square(errors_idw1)))
    print(f"IDW1 RMSE: {rmse_idw1}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW1.")

if errors_idw2:
    rmse_idw2 = numpy.sqrt(numpy.mean(numpy.square(errors_idw2)))
    print(f"IDW2 RMSE: {rmse_idw2}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW2.")

if errors_nn:
    rmse_nn = numpy.sqrt(numpy.mean(numpy.square(errors_nn)))
    print(f"Natural Neighbor RMSE: {rmse_nn}")
else:
    print("No valid interpolated values were found to calculate RMSE for Natural Neighbor.")

if errors_spline:
    rmse_spline = numpy.sqrt(numpy.mean(numpy.square(errors_spline)))
    print(f"Spline RMSE: {rmse_spline}")
else:
    print("No valid interpolated values were found to calculate RMSE for Spline.")

if errors_kriging:
    rmse_kriging = numpy.sqrt(numpy.mean(numpy.square(errors_kriging)))
    print(f"Kriging RMSE: {rmse_kriging}")
else:
    print("No valid interpolated values were found to calculate RMSE for Kriging.")

Iteration 1/50 complete. IDW1 Error: -7.605094909667969, IDW2 Error: -7.771107196807861, NN Error: -8.800724506378174, Spline Error: -8.433640480041504, Kriging Error: -8.257375240325928
Iteration 2/50 complete. IDW1 Error: 7.694990488446473, IDW2 Error: 9.141246172345399, NN Error: 10.294206353581666, Spline Error: 19.78002056733441, Kriging Error: 10.814312818206071
Iteration 3/50 complete. IDW1 Error: -3.851570871652502, IDW2 Error: -4.518502023996252, NN Error: -6.246759203256506, Spline Error: -14.872110632242102, Kriging Error: -4.69929387885465
Iteration 4/50 complete. IDW1 Error: -7.544052149751099, IDW2 Error: -7.413549448945435, NN Error: -2.6893043775343024, Spline Error: -5.663133646943482, Kriging Error: -3.6147003431104743
Iteration 5/50 complete. IDW1 Error: 2.1780834747882256, IDW2 Error: 2.3618174149127373, NN Error: -0.5421800063519111, Spline Error: -5.81072444185301, Kriging Error: -1.42283004983946
Iteration 6/50 complete. IDW1 Error: 6.338254814147949, IDW2 Error:

Iteration 50/50 complete. IDW1 Error: -2.842972981208094, IDW2 Error: -3.217660652869471, NN Error: -2.792774426215418, Spline Error: -7.936352955573328, Kriging Error: -3.77269338297201
IDW1 RMSE: 5.496480232360249
IDW2 RMSE: 5.423151093749088
Natural Neighbor RMSE: 5.366700948892324
Spline RMSE: 9.321633079179211
Kriging RMSE: 5.4252802028821145


*Wind (E-W)*

In [102]:
input_fc = wind_fc_path
errors_idw1 = []
errors_idw2 = []
errors_nn = []
errors_spline = []
errors_kriging = []

# Get all points
points = []
with arcpy.da.SearchCursor(input_fc, ["SHAPE@XY", "wind_e"]) as cursor:
    for row in cursor:
        points.append((row[0], row[1]))

# Randomly sample 50 points
random.seed(42)  # Set seed for reproducibility
sampled_points = random.sample(points, 50)

In [103]:
# Loop over each sampled point, leaving one out each time
for i, (point, actual_value) in enumerate(sampled_points):
    # Create a temporary feature class excluding the current point
    temp_fc = "memory\\temp_points"
    arcpy.management.CreateFeatureclass("memory", "temp_points", "POINT", spatial_reference=input_fc)
    
    # Add the "wind_e" field to the temporary feature class
    arcpy.management.AddField(temp_fc, "wind_e", "DOUBLE")
    
    # Insert all sampled points except the current one
    with arcpy.da.InsertCursor(temp_fc, ["SHAPE@XY", "wind_e"]) as insert_cursor:
        for j, (other_point, other_value) in enumerate(sampled_points):
            if i != j:
                insert_cursor.insertRow([other_point, other_value])
    
    # Perform IDW interpolation with power set to 1
    output_raster_idw1 = arcpy.sa.Idw(temp_fc, "wind_e", power=1.0)
    
    # Perform IDW interpolation with power set to 2
    output_raster_idw2 = arcpy.sa.Idw(temp_fc, "wind_e", power=2.0)
    
    # Perform Natural Neighbor interpolation
    output_raster_nn = arcpy.sa.NaturalNeighbor(temp_fc, "wind_e")
    
    # Perform Spline interpolation
    output_raster_spline = arcpy.sa.Spline(temp_fc, "wind_e")
    
    # Perform Kriging interpolation with spherical model
    output_raster_kriging = arcpy.sa.Kriging(temp_fc, "wind_e", kriging_model="SPHERICAL")
    
    # Extract the raster value at the current point's location
    point_fc = "memory\\temp_point"
    arcpy.management.CreateFeatureclass("memory", "temp_point", "POINT", spatial_reference=input_fc)
    
    with arcpy.da.InsertCursor(point_fc, ["SHAPE@XY"]) as point_cursor:
        point_cursor.insertRow([point])
    
    extracted_values_idw1 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw1, "memory\\extracted_values_idw1")
    extracted_values_idw2 = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_idw2, "memory\\extracted_values_idw2")
    extracted_values_nn = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_nn, "memory\\extracted_values_nn")
    extracted_values_spline = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_spline, "memory\\extracted_values_spline")
    extracted_values_kriging = arcpy.sa.ExtractValuesToPoints(point_fc, output_raster_kriging, "memory\\extracted_values_kriging")
    
    with arcpy.da.SearchCursor(extracted_values_idw1, ["RASTERVALU"]) as cursor_idw1, \
         arcpy.da.SearchCursor(extracted_values_idw2, ["RASTERVALU"]) as cursor_idw2, \
         arcpy.da.SearchCursor(extracted_values_nn, ["RASTERVALU"]) as cursor_nn, \
         arcpy.da.SearchCursor(extracted_values_spline, ["RASTERVALU"]) as cursor_spline, \
         arcpy.da.SearchCursor(extracted_values_kriging, ["RASTERVALU"]) as cursor_kriging:

        for raster_row_idw1, raster_row_idw2, raster_row_nn, raster_row_spline, raster_row_kriging in zip(cursor_idw1, cursor_idw2, cursor_nn, cursor_spline, cursor_kriging):
            interpolated_value_idw1 = raster_row_idw1[0]
            interpolated_value_idw2 = raster_row_idw2[0]
            interpolated_value_nn = raster_row_nn[0]
            interpolated_value_spline = raster_row_spline[0]
            interpolated_value_kriging = raster_row_kriging[0]
            if None not in (interpolated_value_idw1, interpolated_value_idw2, interpolated_value_nn, interpolated_value_spline, interpolated_value_kriging):
                # Calculate the error for each method
                error_idw1 = interpolated_value_idw1 - actual_value
                error_idw2 = interpolated_value_idw2 - actual_value
                error_nn = interpolated_value_nn - actual_value
                error_spline = interpolated_value_spline - actual_value
                error_kriging = interpolated_value_kriging - actual_value
                errors_idw1.append(error_idw1)
                errors_idw2.append(error_idw2)
                errors_nn.append(error_nn)
                errors_spline.append(error_spline)
                errors_kriging.append(error_kriging)
                # Print progress
                print(f"Iteration {i+1}/{len(sampled_points)} complete. IDW1 Error: {error_idw1}, IDW2 Error: {error_idw2}, NN Error: {error_nn}, Spline Error: {error_spline}, Kriging Error: {error_kriging}")
            else:
                print(f"Iteration {i+1}/{len(sampled_points)} complete. Interpolated value is None for one of the methods, skipping this point.")

# Calculate RMSE for all methods
if errors_idw1:
    rmse_idw1 = numpy.sqrt(numpy.mean(numpy.square(errors_idw1)))
    print(f"IDW1 RMSE: {rmse_idw1}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW1.")

if errors_idw2:
    rmse_idw2 = numpy.sqrt(numpy.mean(numpy.square(errors_idw2)))
    print(f"IDW2 RMSE: {rmse_idw2}")
else:
    print("No valid interpolated values were found to calculate RMSE for IDW2.")

if errors_nn:
    rmse_nn = numpy.sqrt(numpy.mean(numpy.square(errors_nn)))
    print(f"Natural Neighbor RMSE: {rmse_nn}")
else:
    print("No valid interpolated values were found to calculate RMSE for Natural Neighbor.")

if errors_spline:
    rmse_spline = numpy.sqrt(numpy.mean(numpy.square(errors_spline)))
    print(f"Spline RMSE: {rmse_spline}")
else:
    print("No valid interpolated values were found to calculate RMSE for Spline.")

if errors_kriging:
    rmse_kriging = numpy.sqrt(numpy.mean(numpy.square(errors_kriging)))
    print(f"Kriging RMSE: {rmse_kriging}")
else:
    print("No valid interpolated values were found to calculate RMSE for Kriging.")

Iteration 1/50 complete. IDW1 Error: 3.0123236179351807, IDW2 Error: 2.246025562286377, NN Error: 2.8740086555480957, Spline Error: 2.14157772064209, Kriging Error: 3.290848970413208
Iteration 2/50 complete. IDW1 Error: -6.4505594092260665, IDW2 Error: -7.202273519600517, NN Error: -7.027326138580925, Spline Error: -9.384787054623253, Kriging Error: -6.877485664452202
Iteration 3/50 complete. IDW1 Error: -7.084330043792724, IDW2 Error: -6.8376302337646475, NN Error: -6.619663200378417, Spline Error: -2.7787713623046866, Kriging Error: -7.431404552459716
Iteration 4/50 complete. IDW1 Error: 21.344667573501926, IDW2 Error: 22.899886508514744, NN Error: 22.72660293154941, Spline Error: 33.5458454089664, Kriging Error: 20.60423888735996
Iteration 5/50 complete. IDW1 Error: 0.5741006254564462, IDW2 Error: 2.557252299727076, NN Error: 5.313239467085475, Spline Error: 16.871374499739282, Kriging Error: 5.737348926008814
Iteration 6/50 complete. IDW1 Error: 5.943902015686035, IDW2 Error: 6.755

Iteration 50/50 complete. IDW1 Error: -0.500870779937248, IDW2 Error: -0.742281035369377, NN Error: 0.7869724474492266, Spline Error: 3.686720772797127, Kriging Error: 1.4269141398015215
IDW1 RMSE: 6.532385869271667
IDW2 RMSE: 6.709311522984089
Natural Neighbor RMSE: 6.993759168852071
Spline RMSE: 9.866407373346306
Kriging RMSE: 6.874853205781382
